In [78]:
import pandas as pd
import re
from PyPDF2 import PdfReader
import os,glob
from pdf2image import convert_from_path
import pytesseract

chris_path = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/PR lab results"
ty_path = r"G:\.shortcut-targets-by-id\1W2cJLOLxbqzOAwJhLhGSqwAfzOjg0nTo\CADAS data upload\PR lab results"

path = chris_path if os.getcwd() == '/Users/chrissoria/Documents/Research/CADAS/ipynb' else ty_path
pdf_path = r"C:\Users\Ty\Downloads\Lab Results 25 Jun 2025.pdf"
csv_path = os.path.join(path, r"All_PDF_Blood_Results.csv") if os.getcwd() == '/Users/chrissoria/Documents/Research/CADAS/ipynb' else r"C:\Users\Ty\Downloads\All_PDF_Blood_Results_PR.csv"

print(path)
print(pdf_path)
print(csv_path)

G:\.shortcut-targets-by-id\1W2cJLOLxbqzOAwJhLhGSqwAfzOjg0nTo\CADAS data upload\PR lab results
C:\Users\Ty\Downloads\Lab Results 25 Jun 2025.pdf
C:\Users\Ty\Downloads\All_PDF_Blood_Results_PR.csv


In [92]:
with open(csv_path, 'r+') as labels:
    labels.write("SA_ClusterID,SA_HouseID,SA_PersonID,SA_Sex,SA_Age,GlobulosBlancos,GlobulosRojos,Hemoglobina,Hematrocrito,VCM,HCM,CHCM,RDW,Plaquetas,VPM,PDW,Neutrofilos,NeutrofilosP,Linfocitos,LinfocitosP,Monocitos,MonocitosP,Eosinofilos,EosinofilosP,Basofilos,BasofilosP,HbA1c,Immature Granulocytes, Immature GranulocytesP, Nucleated RBC, Nucleated RBC P, RDW SD")
with open(pdf_path, 'r') as f:
        reader = PdfReader(pdf_path)
        for i in range(len(reader.pages)//2):
        #for i in range(1):
            finaltext = "start"
            page2 = reader.pages[2*i]
            page1 = reader.pages[2*i+1]
            finaltext = finaltext + "\n" + page2.extract_text() + "\n" + page1.extract_text()
            finaltext = finaltext.replace('\nHi ','')
            finaltext = finaltext.replace('\nLow ','')
            finaltext = finaltext.replace("Hi","")
            finaltext = finaltext.replace("Low","")
            ID_match = re.search(r'(CLUSTER.{0,36})', finaltext)
            if ID_match:
                ParticID = ID_match.group(1)
                ParticID = ParticID.replace("PCTE", ",")
                ParticID = ParticID.replace("PTCE", ",")
                ParticID = ParticID.replace("CLUSTER", ",")
                ParticID = ParticID.replace("CASA", ",")
                cluster = ParticID.split(",")[1]
                cluster = cluster.replace(" ", "")
                houseid = ParticID.split(",")[2]
                houseid = houseid.replace(" ", "")
                personid = ParticID.split(",")[3][0:3]
                personid = personid.replace(" ", "")
                age_present = re.search(r'Y', ParticID)
                if age_present:
                    age_pos = ParticID.index("Y")
                    age = ParticID[(age_pos - 3):age_pos]
                    age = age.strip()
                    sex = ParticID[(age_pos+1):(age_pos+3)]
                    sex = sex.strip()
                    if age[-1] == "y" or age[-1] == "Y":
                        age = age[:-1]
                else:
                    age = "."
                    sex = "."
            else:
                cluster = "."
                houseid = "."
                personid = "."
                age = "."
                sex = "."
            WBC = re.search(r'WBC(.*(\n|\r|\r\n))', finaltext)
            if WBC:
                data1 = WBC[1][5:13]
                data1 = data1.strip()
                data1 = data1.replace(",",".")
                data1 = data1.replace("x","")
            else:
                data1 = "."
            RBC = re.search(r'RBC(.*(\n|\r|\r\n))', finaltext)
            if RBC:
                data2 = RBC[1][5:13]
                data2 = data2.strip()
                data2 = data2.replace(",",".")
                data2 = data2.replace("x","")
            else:
                data2 = "."
            HGB = re.search(r'HGB(.*(\n|\r|\r\n))', finaltext)
            if HGB:
                data3 = HGB[1][5:11]
                data3 = data3.strip()
                data3 = data3.replace(",",".")
            else:
                data3 = "."
            hematocrito = re.search(r'HCT(.*(\n|\r|\r\n))', finaltext)
            if hematocrito:
                data4 = hematocrito[1][5:11]
                data4 = data4.strip()
                data4 = data4.replace(",",".")
            else:
                data4 = "." 
            VCM = re.search(r'MCV(.*(\n|\r|\r\n))', finaltext)
            if VCM:
                data5 = VCM[1][5:9]
                data5 = data5.strip()
                data5 = data5.replace(",",".")
            else:
                data5 = "."
            HCM = re.search(r'MCH(.*(\n|\r|\r\n))', finaltext)
            if HCM:
                data6 = HCM[1][5:9]
                data6 = data6.strip()
                data6 = data6.replace(",",".")
            else:
                data6 = "."
            CHCM = re.search(r'MCHC(.*(\n|\r|\r\n))', finaltext)
            if CHCM:
                data7 = CHCM[1][5:12]
                data7 = data7.strip()
                data7 = data7.replace(",",".")
                data7 = data7.replace("g/","")
            else:
                data7 = "."
            RDW = re.search(r'RDW CV(.*(\n|\r|\r\n))', finaltext)
            if RDW:
                data8 = RDW[1][5:9]
                data8 = data8.strip()
                data8 = data8.replace(",",".")
            else:
                data8 = "."
            plaquetas = re.search(r'PLT(.*(\n|\r|\r\n))', finaltext)
            if plaquetas:
                data9 = plaquetas[1][3:8]
                data9 = data9.strip()
                data9 = data9.replace(",",".")
            else:
                data9 = "."
            VPM = re.search(r'MPV(.*(\n|\r|\r\n))', finaltext)    
            if VPM:
                data10 = VPM[1][5:9]
                data10 = data10.strip()
                data10 = data10.replace(",",".")
            else:
                data10 = "."
            PDW = re.search(r'(PDW...{0,43})', finaltext)
            if PDW:
                data11 = PDW[1][:5]
                data11 = data11.strip()
                data11 = data11.replace(",",".")
            else:
                data11 = "."
            neutrofilos = re.search(r'NEUT #(.*(\n|\r|\r\n))', finaltext)
            if neutrofilos:
                data12 = neutrofilos[1][5:10]
                data12 = data12.strip()
                data12 = data12.replace(",",".")
            else:
                data12 = "."
            neutrofilosP = re.search(r'NEUT %(.*(\n|\r|\r\n))', finaltext)
            if neutrofilosP:
                data13 = neutrofilosP[1][5:10]
                data13 = data13.strip()
                data13 = data13.replace(",",".")
            else:
                data13 = "."
            linfocitos = re.search(r'MCHC(.*(\n|\r|\r\n)){3}', finaltext)
            if linfocitos:
                data14 = linfocitos[1][25:32]
                data14 = data14.strip()
                data14 = data14.replace(",",".")
                data14 = data14.replace("x","")
            else:
                data14 = "."
            linfocitosP = re.search(r'MCHC(.*(\n|\r|\r\n)){2}', finaltext)
            if linfocitosP:
                data15 = linfocitosP[1][19:26]
                data15 = data15.strip()
                data15 = data15.replace(",",".")
            else:
                data15 = "."
            monocitos = re.search(r'MONOCYTES #(.*(\n|\r|\r\n))', finaltext)
            if monocitos:
                data16 = monocitos[1][5:9]
                data16 = data16.strip()
                data16 = data16.replace(",",".")
            else:
                data16 = "."
            monocitosP = re.search(r'MONOCYTES %(.*(\n|\r|\r\n))', finaltext)
            if monocitosP:
                data17 = monocitosP[1][5:9]
                data17 = data17.strip()
                data17 = data17.replace(",",".")
            else:
                data17 = "."
            eosinofilos = re.search(r'EOSINOPHILS #(.*(\n|\r|\r\n))', finaltext)
            if eosinofilos:
                data18 = eosinofilos[1][5:9]
                data18 = data18.strip()
                data18 = data18.replace(",",".")
            else:
                data18 = "."
            eosinofilosP = re.search(r'EOSINOPHILS %(.*(\n|\r|\r\n))', finaltext)
            if eosinofilosP:
                data19 = eosinofilosP[1][5:9]
                data19 = data19.strip()
                data19 = data19.replace(",",".")
            else:
                data19 = "."
            basofilos = re.search(r'BASOPHILS #(.*(\n|\r|\r\n))', finaltext)
            if basofilos:
                data20 = basofilos[1][5:9]
                data20 = data20.strip()
                data20 = data20.replace(",",".")
                data20 = data20.replace("O","0")
            else:
                data20 = "."
            basofilosP = re.search(r'BASOPHILS %(.*(\n|\r|\r\n))', finaltext)
            if basofilosP:
                data21 = basofilosP[1][5:9]
                data21 = data21.strip()
                data21 = data21.replace(",",".")
            else:
                data21 = "."
            SA1cCHECK = re.search(r'SA1c(.*(\n|\r|\r\n))', finaltext)[1][7:11]
            if SA1cCHECK:
                data22 = SA1cCHECK
                data22 = data22.strip()
                data22 = data22.replace(",",".")
            else:
                data22 = "."
            immGranulocytes = re.search(r'IMMATURE GRANULOCYTES #(.*(\n|\r|\r\n))', finaltext)
            if immGranulocytes:
                data23 = immGranulocytes[1][5:9]
                data23 = data23.strip()
                data23 = data23.replace(",",".")
            else:
                data23 = "."
            immGranulocytesP = re.search(r'IMMATURE GRANULOCYTES %(.*(\n|\r|\r\n))', finaltext)
            if immGranulocytesP:
                data24 = immGranulocytesP[1][5:9]
                data24 = data24.strip()
                data24 = data24.replace(",",".")
            else:
                data24 = "."
            nucleatedRBC = re.search(r' NUCLEATED RBC #(.*(\n|\r|\r\n))', finaltext)
            if nucleatedRBC:
                data25 = nucleatedRBC[1][5:10]
                data25 = data25.strip()
                data25 = data25.replace(",",".")
                data25 = data25.replace("O","0")
            else:
                data25 = "."
            nucleatedRBCP = re.search(r'NUCLEATED RBC %(.*(\n|\r|\r\n))', finaltext)
            if nucleatedRBCP:
                data26 = nucleatedRBCP[1][5:10]
                data26 = data26.strip()
                data26 = data26.replace(",",".")
            else:
                data26 = "."
            rdwSD = re.search(r'RDW SD(.*(\n|\r|\r\n))', finaltext)
            if rdwSD:
                data27 = rdwSD[1][5:13]
                data27 = data27.strip()
                data27 = data27.replace(",",".")
                data27 = data27.replace("fL","")
            else:
                data27 = "."
            alldata = "\n" + cluster + "," + houseid + "," + personid + "," + sex + "," + age + "," + data1 + "," + data2 + "," + data3 + "," + data4 + "," + data5 + "," + data6 + "," + data7 + "," + data8 + "," + data9 + "," + data10 + "," + data11 + "," + data12 + "," + data13 + "," + data14 + "," + data15 + "," + data16 + "," + data17 + "," + data18 + "," + data19 + "," + data20 + "," + data21 + "," + data22 + "," + data23 + "," + data24 + "," + data25 + "," + data26 + "," + data27
            with open(csv_path, 'a') as out:
                out.write(alldata)

#Remove duplicate lines
clean = pd.read_csv(csv_path)
clean.drop_duplicates(subset=None, inplace=True)
clean.to_csv(csv_path, index=False)
print("Finished")

Finished


In [94]:
print(finaltext)

start
PRCTRC
LAB-01686
Phone : (787) 759-0306
Fax : 
CLUSTER 52 CASA 75 PCTE 1 16003473 82 Y M 12:43
DR. JIMENEZ Jun/25/2025 Jun/27/2025
Originally Reported Jun/25/2025 09:47:06 pm
Tests Performed by ERM Lic# 3488
Medical Technologist
Lcda. MARIBEL TORRES CAMACHO Lab Director, Lic 2998 - Copyright 2003 Test Performed by IMRL Lic 338CTMPR - Alpha 7SQ2W Control ID - 39231878Hematology
Cell Blood Count
 White Blood Cell Count (WBC)  8.2 x 10^3 /uL 4.8 - 10.8
 Red Blood Cell Count (RBC)    3.9 x 10^6 /uL 4.2 - 6.1
 Hemoglobin(HGB)   11.9 g/dL 14.0 - 18.0
 Hematocrit(HCT)    37.0 (%) 40.1 - 51.0
 Platelet Count(PLT)  192.0 x 10^3 130.0 - 400.0
 MCV   95.9 fL 80.0 - 94.0
 MCH  30.8 pg 27.0 - 31.0
 MCHC   32.2 g/dL 33.0 - 37.0
 Lymphocytes (%)   12.7 (%) 20.0 - 50.0
 Lymphocytes (Lymph)    1.0 x 10^3/uL 1.5 - 5.0
 NEUT %   76.6 (%) 37.0 - 72.0
 NEUT #  6.3 x10^3/uL 1.5 - 7.0
 MONOCYTES %  7.2 (%) 0.0 - 14.0
 MONOCYTES #  0.6 x10^3/uL 0.0 - 0.7
 EOSINOPHILS %  2.8 0.0 - 6.0
 EOSINOPHILS #  0.2

In [58]:
print(ParticID)

, 83 , 10 PTCE 1 16006373 76 Y M 1


In [56]:
print(ID_match)

<re.Match object; span=(53, 96), match='CLUSTER 83 CASA 10 PTCE 1 16006373 76 Y M 1'>
